In [43]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

### Tensor and back prop

In [22]:
x = torch.rand(2, requires_grad=True)
print(x)

tensor([0.2007, 0.8820], requires_grad=True)


In [23]:
y = x*2
print(y)

tensor([0.4014, 1.7641], grad_fn=<MulBackward0>)


In [25]:
grad = torch.tensor([1, 1], dtype=torch.float)
y.backward(grad)
print(x.grad)

tensor([2., 2.])


### NN

In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # the input shape in (<batch size>, 1, 32, 32)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [45]:
input = torch.randn(1, 1, 32, 32)
target = torch.randn(10).view(1, -1) # a dummy target for example

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()

output = net(input)
loss = criterion(output, target)

loss.backward()
optimizer.step()